# Cable Pulling Analysis Tutorial

This notebook demonstrates the basic usage of EasyCablePulling for analyzing cable installation feasibility.

## Learning Objectives

- Load and analyze cable routes from DXF files
- Configure cable and duct specifications
- Interpret analysis results
- Generate reports and visualizations

## Setup

First, let's import the necessary modules and set up our environment.

In [ ]:
import sys
sys.path.append('..')

from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

from easycablepulling.core.models import CableSpec, DuctSpec, CableArrangement
from easycablepulling.core.pipeline import CablePullingPipeline, AnalysisReporter
from easycablepulling.io import load_route_from_dxf

# Configure matplotlib for better plots
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("EasyCablePulling modules imported successfully!")

## 1. Define Cable and Duct Specifications

Before analyzing any route, we need to define the cable and duct specifications.

In [ ]:
# Define a standard single cable
cable_spec = CableSpec(
    diameter=35.0,                    # 35mm diameter
    weight_per_meter=2.5,            # 2.5 kg/m
    max_tension=8000.0,              # 8000N maximum tension
    max_sidewall_pressure=500.0,     # 500 N/m maximum pressure
    min_bend_radius=1200.0,          # 1200mm minimum bend radius
    arrangement=CableArrangement.SINGLE
)

print("Cable Specification:")
print(f"  Diameter: {cable_spec.diameter}mm")
print(f"  Weight: {cable_spec.weight_per_meter}kg/m")
print(f"  Max tension: {cable_spec.max_tension}N")
print(f"  Max pressure: {cable_spec.max_sidewall_pressure}N/m")
print(f"  Min bend radius: {cable_spec.min_bend_radius}mm")

In [ ]:
# Define duct specification
duct_spec = DuctSpec(
    inner_diameter=100.0,            # 100mm inner diameter
    type="PVC",                      # PVC duct material
    friction_dry=0.35,               # Dry friction coefficient
    friction_lubricated=0.15         # Lubricated friction coefficient
)

print("Duct Specification:")
print(f"  Inner diameter: {duct_spec.inner_diameter}mm")
print(f"  Material: {duct_spec.type}")
print(f"  Friction (dry): {duct_spec.friction_dry}")
print(f"  Friction (lubricated): {duct_spec.friction_lubricated}")

# Calculate clearance
clearance = duct_spec.inner_diameter - cable_spec.diameter
clearance_ratio = clearance / cable_spec.diameter
print(f"\nClearance: {clearance}mm ({clearance_ratio:.2f}x cable diameter)")

## 2. Load and Analyze a Simple Route

Let's start with a simple straight route to understand the basic workflow.

In [ ]:
# Create pipeline with conservative settings
pipeline = CablePullingPipeline(
    enable_splitting=True,
    max_cable_length=500.0,         # 500m maximum cable length
    safety_factor=1.5               # 1.5x safety factor
)

print("Pipeline Configuration:")
print(f"  Splitting enabled: {pipeline.enable_splitting}")
print(f"  Max cable length: {pipeline.max_cable_length}m")
print(f"  Safety factor: {pipeline.safety_factor}")

In [ ]:
# Use one of our synthetic test routes
test_route = "../tests/data/straight_route.dxf"

# First, let's examine the route geometry
try:
    original_route = load_route_from_dxf(Path(test_route))
    print(f"Route loaded: {original_route.name}")
    print(f"Number of sections: {len(original_route.sections)}")
    
    for i, section in enumerate(original_route.sections):
        print(f"  Section {section.id}: {len(section.original_polyline)} points, {section.original_length:.1f}m")
        
except Exception as e:
    print(f"Error loading route: {e}")

In [ ]:
# Run complete analysis
result = pipeline.run_analysis(
    test_route,
    cable_spec,
    duct_spec,
    lubricated=False  # Start with dry conditions
)

print("Analysis Results:")
print(f"  Success: {result.success}")
print(f"  Errors: {len(result.errors)}")
print(f"  Warnings: {len(result.warnings)}")

if result.errors:
    for error in result.errors:
        print(f"    Error: {error}")
        
if result.warnings:
    for warning in result.warnings:
        print(f"    Warning: {warning}")

## 3. Interpret Analysis Results

Let's examine the analysis results in detail.

In [ ]:
if result.success or len(result.tension_analyses) > 0:
    # Overall summary
    summary = result.summary
    
    print("Route Summary:")
    print(f"  Total length: {summary['total_length_m']:.1f}m")
    print(f"  Number of sections: {summary['section_count']}")
    
    # Feasibility analysis
    feasibility = summary['feasibility']
    print(f"\nFeasibility Analysis:")
    print(f"  Overall feasible: {feasibility['overall_feasible']}")
    print(f"  Max tension: {feasibility['max_tension_n']:.1f}N")
    print(f"  Max pressure: {feasibility['max_pressure_n_per_m']:.1f}N/m")
    print(f"  Safety factor applied: {feasibility['safety_factor']}")
    
    # Critical sections
    critical = summary['critical_sections']
    if critical['tension_limited'] > 0:
        print(f"  Sections with tension issues: {critical['tension_limited']}")
    if critical['pressure_limited'] > 0:
        print(f"  Sections with pressure issues: {critical['pressure_limited']}")
else:
    print("Analysis failed - cannot display results")

In [ ]:
# Section-by-section details
if result.tension_analyses:
    print("Section-by-Section Analysis:")
    
    for i, tension_analysis in enumerate(result.tension_analyses):
        limit_result = result.limit_results[i]
        
        print(f"\nSection {tension_analysis.section_id}:")
        print(f"  Length: {result.processed_route.sections[i].total_length:.1f}m")
        print(f"  Max tension: {tension_analysis.max_tension:.1f}N")
        print(f"  Max pressure: {limit_result.max_pressure:.1f}N/m")
        print(f"  Recommended direction: {limit_result.recommended_direction}")
        
        # Check limits
        if not limit_result.passes_tension_limit:
            print(f"    ⚠️  Tension limit exceeded!")
        if not limit_result.passes_pressure_limit:
            print(f"    ⚠️  Pressure limit exceeded!")
        if not limit_result.passes_bend_radius_limit:
            print(f"    ⚠️  Bend radius limit exceeded!")
            
        if limit_result.limiting_factors:
            print(f"    Limiting factors: {', '.join(limit_result.limiting_factors)}")

## 4. Visualize Results

Let's create visualizations to better understand the route and analysis results.

In [ ]:
# Plot route geometry
if result.success or len(result.processed_route.sections) > 0:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot 1: Route overview
    for section in result.original_route.sections:
        points = np.array(section.original_polyline)
        ax1.plot(points[:, 0]/1000, points[:, 1]/1000, 'b-', linewidth=2, label='Original route')
    
    # Plot fitted primitives
    for section in result.processed_route.sections:
        for primitive in section.primitives:
            if hasattr(primitive, 'start_point') and hasattr(primitive, 'end_point'):
                # Straight section
                start = np.array(primitive.start_point) / 1000
                end = np.array(primitive.end_point) / 1000
                ax1.plot([start[0], end[0]], [start[1], end[1]], 'r-', linewidth=3, alpha=0.7)
    
    ax1.set_xlabel('Distance (km)')
    ax1.set_ylabel('Distance (km)')
    ax1.set_title('Route Geometry')
    ax1.grid(True, alpha=0.3)
    ax1.axis('equal')
    
    # Plot 2: Tension profile
    if result.tension_analyses:
        for tension_analysis in result.tension_analyses:
            positions = [tr.position for tr in tension_analysis.forward_tensions]
            tensions = [tr.tension for tr in tension_analysis.forward_tensions]
            
            ax2.plot(np.array(positions)/1000, tensions, 'g-', linewidth=2, marker='o', markersize=4)
        
        # Add tension limit line
        ax2.axhline(y=cable_spec.max_tension, color='r', linestyle='--', 
                   label=f'Tension limit ({cable_spec.max_tension}N)')
        
        # Add safety limit line
        safety_limit = cable_spec.max_tension / pipeline.safety_factor
        ax2.axhline(y=safety_limit, color='orange', linestyle='--', 
                   label=f'Safety limit ({safety_limit:.0f}N)')
        
        ax2.set_xlabel('Distance (km)')
        ax2.set_ylabel('Tension (N)')
        ax2.set_title('Tension Profile')
        ax2.grid(True, alpha=0.3)
        ax2.legend()
    
    plt.tight_layout()
    plt.show()
else:
    print("Cannot create visualizations - analysis failed")

## 5. Generate Reports

Let's generate different types of reports from our analysis.

In [ ]:
# Generate text report
if result.success or len(result.tension_analyses) > 0:
    text_report = AnalysisReporter.generate_text_report(result)
    
    # Display first part of report
    report_lines = text_report.split('\n')
    print("\n".join(report_lines[:30]))  # First 30 lines
    
    if len(report_lines) > 30:
        print(f"\n... ({len(report_lines)-30} more lines)")
else:
    print("Cannot generate report - analysis failed")

In [ ]:
# Generate CSV data for further analysis
if result.success or len(result.tension_analyses) > 0:
    csv_data = AnalysisReporter.generate_csv_report(result)
    
    # Display CSV data
    print("CSV Export Data:")
    print(csv_data)
    
    # Could load into pandas for further analysis
    # import pandas as pd
    # from io import StringIO
    # df = pd.read_csv(StringIO(csv_data))
    # print(df.describe())
else:
    print("Cannot generate CSV data - analysis failed")

## 6. Compare Different Scenarios

Let's analyze the same route under different conditions to see the impact.

In [ ]:
# Scenario comparison
scenarios = {
    "Dry, SF=1.5": {"lubricated": False, "safety_factor": 1.5},
    "Lubricated, SF=1.5": {"lubricated": True, "safety_factor": 1.5},
    "Dry, SF=1.0": {"lubricated": False, "safety_factor": 1.0},
    "Lubricated, SF=1.0": {"lubricated": True, "safety_factor": 1.0},
}

scenario_results = {}

for scenario_name, params in scenarios.items():
    # Create pipeline with specific safety factor
    scenario_pipeline = CablePullingPipeline(
        safety_factor=params["safety_factor"],
        max_cable_length=500.0
    )
    
    # Run analysis
    scenario_result = scenario_pipeline.run_analysis(
        test_route,
        cable_spec,
        duct_spec,
        lubricated=params["lubricated"]
    )
    
    scenario_results[scenario_name] = scenario_result
    
    # Print summary
    if scenario_result.success or len(scenario_result.tension_analyses) > 0:
        feasible = scenario_result.summary.get('feasibility', {}).get('overall_feasible', False)
        max_tension = scenario_result.summary.get('feasibility', {}).get('max_tension_n', 0)
        print(f"{scenario_name:20}: {'✓ Feasible' if feasible else '✗ Not feasible'} (max tension: {max_tension:.0f}N)")
    else:
        print(f"{scenario_name:20}: ✗ Analysis failed")

In [ ]:
# Visualize scenario comparison
fig, ax = plt.subplots(figsize=(12, 6))

scenario_names = []
max_tensions = []
feasible_status = []

for name, scenario_result in scenario_results.items():
    if scenario_result.success or len(scenario_result.tension_analyses) > 0:
        scenario_names.append(name)
        max_tension = scenario_result.summary.get('feasibility', {}).get('max_tension_n', 0)
        max_tensions.append(max_tension)
        feasible = scenario_result.summary.get('feasibility', {}).get('overall_feasible', False)
        feasible_status.append(feasible)

if scenario_names:
    # Create bar chart
    colors = ['green' if feasible else 'red' for feasible in feasible_status]
    bars = ax.bar(scenario_names, max_tensions, color=colors, alpha=0.7)
    
    # Add tension limit line
    ax.axhline(y=cable_spec.max_tension, color='red', linestyle='-', linewidth=2, 
               label=f'Cable limit ({cable_spec.max_tension}N)')
    
    ax.set_ylabel('Maximum Tension (N)')
    ax.set_title('Scenario Comparison: Maximum Tension')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Rotate x-axis labels for readability
    plt.xticks(rotation=45, ha='right')
    
    # Add value labels on bars
    for bar, tension in zip(bars, max_tensions):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 100,
                f'{tension:.0f}N', ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.show()
else:
    print("No successful scenarios to compare")

## 7. Analyze Different Route Types

Let's compare analysis results for different route geometries.

In [ ]:
# Test different route types
route_types = {
    "Straight": "../tests/data/straight_route.dxf",
    "S-Curve": "../tests/data/s_curve_route.dxf",
    "90° Arc": "../tests/data/circular_arc_90.0deg.dxf",
    "Complex": "../tests/data/complex_route.dxf"
}

# Use consistent pipeline settings
comparison_pipeline = CablePullingPipeline(
    safety_factor=1.0,  # No extra safety margin for comparison
    max_cable_length=1000.0
)

route_results = {}

for route_name, route_file in route_types.items():
    try:
        route_result = comparison_pipeline.run_analysis(
            route_file,
            cable_spec,
            duct_spec,
            lubricated=True  # Use lubrication for better comparison
        )
        route_results[route_name] = route_result
        
        # Print summary
        if route_result.success or len(route_result.tension_analyses) > 0:
            max_tension = route_result.summary.get('feasibility', {}).get('max_tension_n', 0)
            total_length = route_result.summary.get('total_length_m', 0)
            feasible = route_result.summary.get('feasibility', {}).get('overall_feasible', False)
            
            print(f"{route_name:10}: {total_length:6.1f}m, {max_tension:7.0f}N {'✓' if feasible else '✗'}")
        else:
            print(f"{route_name:10}: Analysis failed")
            
    except Exception as e:
        print(f"{route_name:10}: Error - {e}")

In [ ]:
# Create comparison visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Extract data for plotting
route_names = []
lengths = []
tensions = []
colors = []

for name, route_result in route_results.items():
    if route_result.success or len(route_result.tension_analyses) > 0:
        route_names.append(name)
        lengths.append(route_result.summary.get('total_length_m', 0))
        tensions.append(route_result.summary.get('feasibility', {}).get('max_tension_n', 0))
        feasible = route_result.summary.get('feasibility', {}).get('overall_feasible', False)
        colors.append('green' if feasible else 'red')

if route_names:
    # Plot 1: Route lengths
    ax1.bar(route_names, lengths, color='skyblue', alpha=0.7)
    ax1.set_ylabel('Route Length (m)')
    ax1.set_title('Route Lengths by Type')
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Maximum tensions
    bars = ax2.bar(route_names, tensions, color=colors, alpha=0.7)
    ax2.axhline(y=cable_spec.max_tension, color='red', linestyle='-', linewidth=2,
                label=f'Cable limit ({cable_spec.max_tension}N)')
    ax2.set_ylabel('Maximum Tension (N)')
    ax2.set_title('Maximum Tension by Route Type')
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    
    # Add value labels
    for bar, tension in zip(bars, tensions):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height + 200,
                f'{tension:.0f}N', ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.show()
else:
    print("No successful analyses to compare")

## 8. Explore Advanced Features

Let's explore some advanced features like route splitting and trefoil arrangements.

In [ ]:
# Test route splitting on a long route
long_route = "../tests/data/long_route.dxf"

# Compare with and without splitting
splitting_comparison = {}

# Without splitting
no_split_pipeline = CablePullingPipeline(
    enable_splitting=False,
    safety_factor=1.0
)

try:
    no_split_result = no_split_pipeline.run_analysis(long_route, cable_spec, duct_spec, lubricated=True)
    splitting_comparison["No splitting"] = no_split_result
except Exception as e:
    print(f"No splitting analysis failed: {e}")

# With splitting
split_pipeline = CablePullingPipeline(
    enable_splitting=True,
    max_cable_length=400.0,
    safety_factor=1.0
)

try:
    split_result = split_pipeline.run_analysis(long_route, cable_spec, duct_spec, lubricated=True)
    splitting_comparison["With splitting"] = split_result
except Exception as e:
    print(f"Splitting analysis failed: {e}")

# Compare results
print("Splitting Comparison:")
for approach, approach_result in splitting_comparison.items():
    if approach_result.success or len(approach_result.tension_analyses) > 0:
        sections = approach_result.summary.get('section_count', 0)
        max_tension = approach_result.summary.get('feasibility', {}).get('max_tension_n', 0)
        feasible = approach_result.summary.get('feasibility', {}).get('overall_feasible', False)
        
        print(f"  {approach:15}: {sections} sections, max tension {max_tension:.0f}N {'✓' if feasible else '✗'}")
    else:
        print(f"  {approach:15}: Analysis failed")

In [ ]:
# Test trefoil cable arrangement
trefoil_cable = CableSpec(
    diameter=50.0,                   # Larger diameter
    weight_per_meter=4.0,           # Heavier
    max_tension=15000.0,            # Higher tension capacity
    max_sidewall_pressure=800.0,    # Higher pressure capacity
    min_bend_radius=1500.0,         # Larger bend radius
    arrangement=CableArrangement.TREFOIL,
    number_of_cables=3
)

# Larger duct for trefoil
large_duct = DuctSpec(
    inner_diameter=150.0,           # Larger duct
    type="PVC",
    friction_dry=0.35,
    friction_lubricated=0.15
)

# Test trefoil on S-curve route
s_curve_route = "../tests/data/s_curve_route.dxf"

trefoil_pipeline = CablePullingPipeline(safety_factor=1.0)

try:
    trefoil_result = trefoil_pipeline.run_analysis(
        s_curve_route,
        trefoil_cable,
        large_duct,
        lubricated=True
    )
    
    print("Trefoil Cable Analysis:")
    if trefoil_result.success or len(trefoil_result.tension_analyses) > 0:
        print(f"  Cable arrangement: {trefoil_cable.arrangement.value}")
        print(f"  Number of cables: {trefoil_cable.number_of_cables}")
        print(f"  Duct clearance: {large_duct.inner_diameter - trefoil_cable.diameter:.1f}mm")
        
        feasible = trefoil_result.summary.get('feasibility', {}).get('overall_feasible', False)
        max_tension = trefoil_result.summary.get('feasibility', {}).get('max_tension_n', 0)
        max_pressure = trefoil_result.summary.get('feasibility', {}).get('max_pressure_n_per_m', 0)
        
        print(f"  Feasible: {'✓' if feasible else '✗'}")
        print(f"  Max tension: {max_tension:.1f}N")
        print(f"  Max pressure: {max_pressure:.1f}N/m")
    else:
        print("  Analysis failed")
        
except Exception as e:
    print(f"Trefoil analysis error: {e}")

## 9. Export Results

Finally, let's export our results in various formats for further use.

In [ ]:
# Create output directory
output_dir = Path("tutorial_outputs")
output_dir.mkdir(exist_ok=True)

# Export results from our main analysis
if result.success or len(result.tension_analyses) > 0:
    
    # Save text report
    text_report = AnalysisReporter.generate_text_report(result)
    with open(output_dir / "analysis_report.txt", "w") as f:
        f.write(text_report)
    print(f"✓ Saved text report: {output_dir / 'analysis_report.txt'}")
    
    # Save CSV data
    csv_data = AnalysisReporter.generate_csv_report(result)
    with open(output_dir / "section_data.csv", "w") as f:
        f.write(csv_data)
    print(f"✓ Saved CSV data: {output_dir / 'section_data.csv'}")
    
    # Save JSON summary
    import json
    json_summary = AnalysisReporter.generate_json_summary(result)
    with open(output_dir / "analysis_summary.json", "w") as f:
        json.dump(json_summary, f, indent=2)
    print(f"✓ Saved JSON summary: {output_dir / 'analysis_summary.json'}")
    
    # Save scenario comparison
    scenario_data = {}
    for name, scenario_result in scenario_results.items():
        if scenario_result.success or len(scenario_result.tension_analyses) > 0:
            scenario_data[name] = {
                "feasible": scenario_result.summary.get('feasibility', {}).get('overall_feasible', False),
                "max_tension_n": scenario_result.summary.get('feasibility', {}).get('max_tension_n', 0),
                "total_length_m": scenario_result.summary.get('total_length_m', 0)
            }
    
    with open(output_dir / "scenario_comparison.json", "w") as f:
        json.dump(scenario_data, f, indent=2)
    print(f"✓ Saved scenario comparison: {output_dir / 'scenario_comparison.json'}")
    
else:
    print("No results to export")

print(f"\nAll outputs saved to: {output_dir.absolute()}")

## Summary

This tutorial covered:

1. **Basic setup**: Defining cable and duct specifications
2. **Route analysis**: Running complete cable pulling analysis
3. **Result interpretation**: Understanding feasibility and limitations
4. **Visualization**: Creating plots to understand results
5. **Scenario comparison**: Comparing different analysis conditions
6. **Advanced features**: Route splitting and trefoil arrangements
7. **Export options**: Saving results in multiple formats

## Next Steps

- Try the advanced geometry processing tutorial
- Explore professional visualization options
- Learn about batch processing workflows
- Read the API reference for detailed function documentation